In [1]:
# ---------------------------------------
# Step 1: Read from Bronze Lakehouse
# ---------------------------------------

# Replace this with your actual path from the Bronze lakehouse
bronze_path = "abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Enterprise_datawarehouse_lakehouse_bronze.Lakehouse/Files/Raw_data_bronze/Product.csv"  # Example, paste your actual path here

# Read the CSV with proper options
df_bronze = (
    spark.read.format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .option("escape", "\\")  # Handles backslashes like in 'Black\, 58'
    .load(bronze_path)
)

display(df_bronze)  # Optional: Preview

# ---------------------------------------
# Step 2: Simple Cleaning
# ---------------------------------------

# Rename columns for consistency (example)
df_cleaned = (
    df_bronze
    .withColumnRenamed("ProductKey", "product_key")
    .withColumnRenamed("Product", "product_name")
    .withColumnRenamed("Standard Cost", "standard_cost")
    .withColumnRenamed("Color", "color")
    .withColumnRenamed("Subcategory", "subcategory")
    .withColumnRenamed("Category", "category")
    .withColumnRenamed("Background Color Format", "bg_color_format")
    .withColumnRenamed("Font Color Format", "font_color_format")
)

# Drop rows with nulls
df_cleaned = df_cleaned.dropna()

# ---------------------------------------
# Step 3: Write to Silver Lakehouse as Delta Table
silver_table_path = "Tables/product_delta"

df_cleaned.write.format("delta").mode("overwrite").save(silver_table_path)

# Step 1: Read the Delta table back (even though we just wrote it)
product_delta_df = spark.read.format("delta").load("Tables/product_delta")

# Step 2: Register the table to dbo schema using AS SELECT
product_delta_df.createOrReplaceTempView("product_temp_view")

spark.sql("""
    CREATE OR REPLACE TABLE dbo.product_delta
    USING DELTA
    AS SELECT * FROM product_temp_view
""")


print("Delta table written successfully to Silver Lakehouse.")



StatementMeta(, 0278cbc0-2aa0-4943-85bf-37f027b1cdd4, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c590716b-eff9-4ff2-9e0b-8870171cc42c)

Delta table written successfully to Silver Lakehouse.


In [2]:
# ---------------------
# Region CSV → Delta Table
# ---------------------

# Step 1: Read Region data from Bronze Lakehouse
region_path = "abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Enterprise_datawarehouse_lakehouse_bronze.Lakehouse/Files/Raw_data_bronze/Region.csv"
region_df = spark.read.option("header", True).option("delimiter", "\t").csv(region_path)

# Step 2: Clean region data
region_cleaned = region_df \
    .withColumnRenamed("SalesTerritoryKey", "sales_territory_key") \
    .withColumnRenamed("Region", "region") \
    .withColumnRenamed("Country", "country") \
    .withColumnRenamed("Group", "region_group") \
    .dropna()

# Step 3: Write Region data as Delta table to Silver Lakehouse
region_cleaned.write.format("delta").mode("overwrite").save("Tables/region_delta")

# Step 1: Read the Delta table back (even though we just wrote it)
region_delta_df = spark.read.format("delta").load("Tables/region_delta")

# Step 2: Register the table to dbo schema using AS SELECT
region_delta_df.createOrReplaceTempView("region_temp_view")

spark.sql("""
    CREATE OR REPLACE TABLE dbo.region_delta
    USING DELTA
    AS SELECT * FROM region_temp_view
""")


StatementMeta(, 0278cbc0-2aa0-4943-85bf-37f027b1cdd4, 4, Finished, Available, Finished)

DataFrame[]

In [3]:
# ---------------------
# Sales CSV → Delta Table
# ---------------------

# Step 1: Read Sales data from Bronze Lakehouse
sales_path = "abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Enterprise_datawarehouse_lakehouse_bronze.Lakehouse/Files/Raw_data_bronze/Sales.csv"
sales_df = spark.read.option("header", True).option("delimiter", ",").csv(sales_path)

# Step 2: Clean sales data
sales_cleaned = sales_df \
    .withColumnRenamed("SalesOrderNumber", "sales_order_number") \
    .withColumnRenamed("OrderDate", "order_date") \
    .withColumnRenamed("ProductKey", "product_key") \
    .withColumnRenamed("SalesTerritoryKey", "sales_territory_key") \
    .withColumnRenamed("OrderQuantity", "order_quantity") \
    .withColumnRenamed("UnitPrice", "unit_price") \
    .withColumnRenamed("SalesAmount", "sales_amount") \
    .withColumnRenamed("TotalProductCost", "total_product_cost") \
    .dropna()

# Step 3: Write Sales data as Delta table to Silver Lakehouse
sales_cleaned.write.format("delta").mode("overwrite").save("Tables/sales_delta")

# Step 1: Read the Delta table back (even though we just wrote it)
sales_delta_df = spark.read.format("delta").load("Tables/sales_delta")

# Step 2: Register the table to dbo schema using AS SELECT
sales_delta_df.createOrReplaceTempView("sales_temp_view")

spark.sql("""
    CREATE OR REPLACE TABLE dbo.sales_delta
    USING DELTA
    AS SELECT * FROM sales_temp_view
""")



StatementMeta(, 0278cbc0-2aa0-4943-85bf-37f027b1cdd4, 5, Finished, Available, Finished)

DataFrame[]